In [2]:
!pip install pandas==1.3.5
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
from keras.layers import RNN
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import pickle
import os
from numpy.random import choice
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

with open(f"/content/drive/My Drive/Colab Notebooks/cpt_2_data.p", "rb") as f2:
    data = pickle.load(f2)

DATASIZE = 10

data = data.loc[0:DATASIZE-1]['cpt_input']

text = " ".join(data.to_numpy())

characters = sorted(list(set(" ".join(text)+'`')))
vocab_size = len(characters)

X = []   # extracted sequences
Y = []   # the target - the follow up character

seq_length = 180   #number of characters to consider before predicting the following character

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

length = len(text)

for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
    
print('Number of extracted sequences:', len(X))
    
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

model = Sequential()
model.add(GRU(600, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(600))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

filepath="model_weights/baseline-ton-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X_modified, Y_modified, epochs=6, batch_size=256, callbacks = callbacks_list)

model.save("LSTM_trained")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of extracted sequences: 2312
Epoch 1/6
10/10 [==============================] - ETA: 0s - loss: 3.7437 - accuracy: 0.1354
Epoch 00001: loss improved from inf to 3.74371, saving model to model_weights/baseline-ton-01-3.7437.hdf5
10/10 [==============================] - 10s 628ms/step - loss: 3.7437 - accuracy: 0.1354
Epoch 2/6
10/10 [==============================] - ETA: 0s - loss: 3.4557 - accuracy: 0.0748
Epoch 00002: loss improved from 3.74371 to 3.45574, saving model to model_weights/baseline-ton-02-3.4557.hdf5
10/10 [==============================] - 6s 621ms/step - loss: 3.4557 - accuracy: 0.0748
Epoch 3/6
10/10 [==============================] - ETA: 0s - loss: 3.1554 - accuracy: 0.1648
Epoch 00003: loss improved from 3.45574 to 3.15544, saving model to model_weights/baseline-ton-03-3.1554.hdf5
10/10 [==============================] - 6s 618ms/s

INFO:tensorflow:Assets written to: LSTM_trained/assets


INFO:tensorflow:Assets written to: LSTM_trained/assets
